In [16]:
import pandas as pd
import json
priceData = 'priceData/aave_price_data.csv'
jsonFile = 'sentimentalOutput/discordGroup1.json'
def calculateAccuracy(priceData,jsonFile):
    monthlyPrice = pd.read_csv(priceData)
    monthlyPrice = monthlyPrice[['Date','Price']]
    monthlyPrice['Date'] = pd.to_datetime(monthlyPrice['Date'])
    monthlyPrice['changePrice'] = monthlyPrice['Price'].pct_change(fill_method ='ffill')
    with open(jsonFile) as json_file:
        data = json.load(json_file)
    date = []
    senti = []
    for key in data:
        date.append(key)
        senti.append(data[key])
    s1={'Date':date,'sentiment':senti}
    s1 = pd.DataFrame(s1)
    
    s1["Date"] = pd.to_datetime(s1['Date'])
    s1['changeSen'] = s1['sentiment'].pct_change(fill_method ='ffill')
    s1['changeSen'] = s1['changeSen'][::-1].shift(periods=1)
    print(s1)
    new = s1.merge(monthlyPrice, on='Date')
    compare = new[['changeSen','changePrice']]
    compare = compare.dropna()
    compare['changeSen'] = [True if x > 0 else False for x in compare['changeSen']]
    compare['changePrice'] = [True if x > 0 else False for x in compare['changePrice']]
    compare['res'] = compare['changePrice'] == compare['changeSen']
    accuracy = compare['res'].mean()
print(calculateAccuracy(priceData,jsonFile))

         Date  sentiment  changeSen
0  2021-01-01   0.125934  -0.109870
1  2021-02-01   0.112097   0.049781
2  2021-03-01   0.117678   0.091044
3  2021-04-01   0.128392  -0.155429
4  2021-05-01   0.108436   0.243923
5  2021-06-01   0.134886   0.189940
6  2021-07-01   0.160506  -0.060958
7  2021-08-01   0.150722  -0.053700
8  2021-09-01   0.142628   0.016831
9  2021-10-01   0.145029  -0.153827
10 2021-11-01   0.122719  -0.082414
11 2021-12-01   0.112605  -0.017533
12 2022-01-01   0.110631   0.229142
13 2022-02-01   0.135981   0.075208
14 2022-03-01   0.146208        NaN
None
